In [5]:
import numpy as np

## Ball to Handler

In [9]:
def length(p1, p2, axis=-1):
    return np.sqrt((p1[...,0]-p2[...,0])**2+(p1[...,1]-p2[...,1])**2)

MODELS = [
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D-no-feat/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/1G3D3P/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/CVAE/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/noSN/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/acc_pen/results/reconstruct.npy',
    '/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy'
]

In [10]:
for m_path in MODELS:
    if m_path == '/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy':
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/rfeat_valid.npy'
    else:
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/feat_valid.npy'
    rec_data = np.load(m_path)
    rec_data = np.reshape(rec_data, [rec_data.shape[0], rec_data.shape[1], 11, 2])
    num_data,num_seq,num_player,num_feat = rec_data.shape
    feat_data = np.load(feat_path)[:,:,:5] # [?,?,6] -> [?,?,5] remove basket feature
    # mask to filter out ball passing  
    mask = np.sum(feat_data, axis=-1)!=0
    feat_data = np.argmax(feat_data, axis=-1) + 1 # ball:0, offense:1-5, defense:6-10
    # get index
    idx = np.repeat(np.repeat(np.arange(num_data), [num_seq]),[2])
    t_idx = np.repeat(np.tile(np.arange(num_seq), [num_data]),[2])
    feat_idx = np.repeat(np.reshape(feat_data, [-1,]),[2])
    xy_idx = np.tile(np.arange(2), [num_data*num_seq])
    # gathering
    handler_pos = rec_data[idx,t_idx,feat_idx,xy_idx].reshape([num_data, num_seq, num_feat])
    ball_pos = rec_data[:,:,0,:]
    # compute distance
    lens = length(handler_pos, ball_pos)
    dist = np.sum(lens*mask)/np.count_nonzero(mask)
    std = (np.sum((lens-dist)**2*mask)/(np.count_nonzero(mask)-1))**0.5
    print('{0:100}:  {1:.4f}, std:{2:.4f}'.format(m_path, dist, std))

/workspace/data/nctu_cgvlab_bballgan/Log/1G3D-no-feat/results/reconstruct.npy                       :  1.8084, std:1.2922
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D/results/reconstruct.npy                               :  1.5991, std:1.1149
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D3P/results/reconstruct.npy                             :  1.3632, std:0.9749
/workspace/data/nctu_cgvlab_bballgan/Log/CVAE/reconstruct.npy                                       :  1.2063, std:1.0166
/workspace/data/nctu_cgvlab_bballgan/Log/noSN/results/reconstruct.npy                               :  1.5292, std:1.2204
/workspace/data/nctu_cgvlab_bballgan/Log/acc_pen/results/reconstruct.npy                            :  1.5335, std:1.0931
/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy                                             :  1.5537, std:0.7944


## Ball to Defender

In [11]:
for m_path in MODELS:
    if m_path == '/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy':
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/rfeat_valid.npy'
    else:
        feat_path = '/workspace/data/nctu_cgvlab_bballgan/Log/feat_valid.npy'   
    rec_data = np.load(m_path)
    rec_data = np.reshape(rec_data, [rec_data.shape[0], rec_data.shape[1], 11, 2])
    num_data,num_seq,num_player,num_feat = rec_data.shape
    feat_data = np.load(feat_path)[:,:,:5] # [?,?,6] -> [?,?,5] remove basket feature
    # mask to filter out ball passing  
    mask = np.sum(feat_data, axis=-1)!=0
    defender_pos = rec_data[:,:,6:11,:]
    ball_pos = rec_data[:,:,0:1,:]
    # compute distance
    def_5lens = length(defender_pos, ball_pos)
    def_best = np.amin(def_5lens, axis=-1)
    dist =  np.sum(def_best*mask)/np.count_nonzero(mask)
    std = (np.sum((def_best-dist)**2*mask)/(np.count_nonzero(mask)-1))**0.5
    print('{0:100}:  {1:.4f}, std:{2:.4f}'.format(m_path, dist, std))

/workspace/data/nctu_cgvlab_bballgan/Log/1G3D-no-feat/results/reconstruct.npy                       :  6.7745, std:3.3128
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D/results/reconstruct.npy                               :  6.8084, std:3.2307
/workspace/data/nctu_cgvlab_bballgan/Log/1G3D3P/results/reconstruct.npy                             :  6.5078, std:3.2255
/workspace/data/nctu_cgvlab_bballgan/Log/CVAE/reconstruct.npy                                       :  6.1882, std:2.2477
/workspace/data/nctu_cgvlab_bballgan/Log/noSN/results/reconstruct.npy                               :  6.3518, std:2.7743
/workspace/data/nctu_cgvlab_bballgan/Log/acc_pen/results/reconstruct.npy                            :  6.6968, std:3.0402
/workspace/data/nctu_cgvlab_bballgan/Log/data_valid.npy                                             :  6.3794, std:2.7448
